In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../") # go to parent dir

In [3]:
import gym
from gym import spaces
import numpy as np
import sciunit
import scipy

In [15]:
spaces.Discrete

gym.spaces.discrete.Discrete

In [9]:
class A():
    a = 123
    
    def print_a(self):
        print(A.a)
        
    def touch_a(self):
        A.a = 124


test=A()
test.a
test.touch_a()
test.print_a()

124


In [10]:
A.a

124

In [17]:
a = spaces.MultiBinary(3).sample()

In [4]:
from src.ldmunit.models.RWCK import RWCKModel

In [5]:
from bandit import BanditTwoArmedHighLowFixed

In [6]:
paras = dict(zip(['alpha', 'alpha_c', 'beta', 'beta_c', 'w0'], [1,1,1,1,0]))
model = RWCKModel(2, 1, paras= paras)
env = BanditTwoArmedHighLowFixed()

In [7]:
def simulate(env, model, n_trials, seed=0):
    """Simulation in a given AI Gym environment."""
    #TODO: add support for env.seed()
    assert isinstance(env, gym.Env)
    assert isinstance(model, sciunit.Model)
    assert isinstance(n_trials, int)
    
    assert model.paras != None #TODO: add assert for keys


    # reset the agent state and 
    model.reset()
    init_stimulus = env._reset()

    a = np.zeros(n_trials, dtype=int) #TODO: change dtype
    r = np.zeros(n_trials, dtype=int)
    s = np.zeros(n_trials, dtype=int)

    # add the first stimulus in the environment
    s = np.insert(s, 0, init_stimulus, axis=0)

    for i in range(n_trials):
        # compute choice probabilities
        P = model.predict(s[i])

        # action based on choice probabilities
        a[i] = model.act(P)

        # generate reward based on action
        s[i+1], r[i], done, _ = env._step(a[i])

        # update choice kernel and Q weights
        model.update(s[i], r[i], a[i], done)

    # delete the extra stimulus
    s = np.delete(s, n_trials, axis=0)

    env.close()

    obs = {
        'stimuli': s,
        'actions': a,
        'rewards': r,
    }

    return obs

def simulate_multi(env, model, n_trials, seed=0):
    """Simulation in a given AI Gym environment."""
    #TODO: add support for env.seed()
    assert isinstance(env, gym.Env)
    assert isinstance(model, sciunit.Model)
    assert isinstance(n_trials, int)
    
    assert model.paras != None #TODO: add assert for keys
    
    model.reset()
    actions, stimuli, rewards = [], [], []
    
    for j in range(model.n_sub):
        # reset the agent state and 
        init_stimulus = env._reset()

        a = np.zeros(n_trials, dtype=int) #TODO: change dtype
        r = np.zeros(n_trials, dtype=int)
        s = np.zeros(n_trials, dtype=int)

        # add the first stimulus in the environment
        s = np.insert(s, 0, init_stimulus, axis=0)
    
        for i in range(n_trials):
            # compute choice probabilities
            P = model.predict(s[i], j)

            # action based on choice probabilities
            a[i] = model.act(P)

            # generate reward based on action
            s[i+1], r[i], done, _ = env._step(a[i])

            # update choice kernel and Q weights
            model.update(s[i], r[i], a[i], done, j)

        # delete the extra stimulus
        s = np.delete(s, n_trials, axis=0)
        
        actions.append(a)
        stimuli.append(s)
        rewards.append(r)

        env.close()

    obs = {
        'stimuli': stimuli,
        'actions': actions,
        'rewards': rewards,
    }

    return obs

In [8]:
def loglike(model, stimuli, rewards, actions):
    #TODO: add assertion for the length
    n_trials = len(stimuli)
    
    res = 0
    
    # reset the model's state
    model.reset()
    done = False
    
    for i in range(n_trials):
        # compute choice probabilities
        P = model.predict(stimuli[i])
        
        # probability of the action
        #TODO: generalize this
        p = model.loglikelihood(P, actions[i])

        # add log-likelihood
        res += np.log(p)
        # update choice kernel and Q weights
        model.update(stimuli[i], rewards[i], actions[i], done)
    
    return res

In [9]:
def fit(model, stimuli, rewards, actions, fixed):
    
    vals = list(fixed.values())

    def objective_fun(vals):
        for k, v in zip(fixed.keys(), vals):
            model.paras[k] = v
        return -loglike(model, stimuli, rewards, actions)

    opt_results = scipy.optimize.minimize(fun=objective_fun,
                                          x0=vals, 
                                          bounds=[(0,1000)]*len(vals))
    if opt_results.success:
        for k, v in zip(fixed.keys(), opt_results.x):
            model.paras[k] = v
    return opt_results

In [10]:
res = simulate(env, model, 100)

In [11]:
a = loglike(model, res['stimuli'], res['rewards'], res['actions'])
print("Log-likehood value", a)

Log-likehood value -42.59922642295281


In [12]:
print("paras before minimize", model.paras)

paras before minimize {'alpha': 1, 'alpha_c': 1, 'beta': 1, 'beta_c': 1, 'w0': 0}


In [13]:
fixed = dict(zip(['beta_c'], [1]))
opt = fit(model, res['stimuli'], res['rewards'], res['actions'], fixed)

In [14]:
print("paras after minimize", model.paras)

paras after minimize {'alpha': 1, 'alpha_c': 1, 'beta': 1, 'beta_c': 2.258099102131004, 'w0': 0}


In [15]:
from src.ldmunit.models.RWCK import RWCKMultiModel

In [24]:
multi_model = RWCKMultiModel(2, 1, paras=[paras]*2)

In [25]:
simulate_multi(env, multi_model, 100)

{'stimuli': [array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
  array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])],
 'actions': [array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0,
         0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0

In [26]:
from src.ldmunit.models.nwsls import NWSLSModel

In [27]:
nw = NWSLSModel(2,1, dict(epsilon=0.5))

In [28]:
simulate(env, nw, 100)

{'stimuli': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'actions': array([1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0,
        0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1,
        0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1,
        1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1]),
 'rewards': array([0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
        1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1,
        1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,

In [29]:
from src.ldmunit.models.nwsls import NWSLSMultiModel

In [30]:
multi_nw = NWSLSMultiModel(2,1, [dict(epsilon=0.5)]*2)

In [31]:
simulate_multi(env, multi_nw, 100)

{'stimuli': [array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
  array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])],
 'actions': [array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1,
         0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1,
         1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0,
         1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0

In [33]:
a = loglike(nw, res['stimuli'], res['rewards'], res['actions'])
print("Log-likehood value", a)

Log-likehood value -57.73759185865701


In [34]:
a = loglike(model, res['stimuli'], res['rewards'], res['actions'])
print("Log-likehood value", a)

Log-likehood value -40.29928461375536
